# On-policy Control with Approximation



Tabular approach to approximate value functions can't handle RL problems with large/continuous state-space. To achieve this, any of a broad range of existing methods for supervised-learning function approximation can be used simply by treating each backup as a training example. Parameterized function approximation can very easily be used here. 

The approximate value function is represented as a parameterized functional form with weight vector $\theta \in \mathbb{R}^n$. Although the weight vector has many components (n), $|\mathbb{S}| >> n$ and we must settle for an approximate solution. The parametric approximation of the action-value function $\hat{q}(s,a,\theta) \approx q^{*}(s,a)$. We can use MSE($\theta$) as a measure of the error in the value function. The optimal value function can be achieved using stochastic gradient descent (SGD) on weight vector.

General SGD is : $$ \theta_{t+1} \leftarrow \theta_{t} - \alpha \nabla_{\theta}(Error_t)^2 $$
where Error in case of state-action value function can be represented as -
$$Error_t = Target_t - \hat{q}(s,a,\theta)$$


## Semi-gradient Methods

Target value can be computed using a lot of different ways like MC, TD(0), n-Step return etc. Unless target value is computed using MC methods, bootstrapping from current estimate of weight vector $\theta$ is needed to approximate future rewards. Therefore, for all methods except MC, target value is a function of weight vector. True gradient of Error is complex to evaluate and doesn't offer much benefit over the alternative - Semi-gradient methods. 

In Semi-gradient TD methods, the weight vector appears in the update target, yet this is not taken into account in computing the gradient thus the name. As such, they cannot rely on classical SGD results. Nevertheless, good results can be obtained for semi-gradient methods in the special case of linear function approximation, in which the value estimates are $\phi.\theta$ .

## Feature Selection
Choosing the features is one of the most important ways of adding prior domain knowledge to reinforcement learning systems. Features can be chosen as polynomials, but this case generalizes poorly in the online setting. Features can also be based on Fourier basis functions, Radial basis functions or some form of coarse coding. Tile coding is a form of coarse coding that is particularly computationally efficient and flexible.

## Tile Coding
In tile coding the receptive fields of the features are grouped into partitions of the input space. Each such partition is called a tiling, and each element of the partition is called a tile. Feature of a state in case of tile coding is the tile in which the state falls; generalization would be have overlapping receptive field. To get true coarse coding with tile coding, multiple tilings are needed, each offset by a fraction of a tile width. Every state falls in exactly one tile in each of the 'n' tilings. These 'n' tiles correspond to n features that become active when the state occurs. Therefore, the feature vector $\phi(s)$ has one component for each tile in
each tiling. 


## Experiment

Here, I experiment with Mountain Car domain. Aim is to get an under powered car to the top of a hill (top = 0.5 position). The car is on a one-dimensional track, positioned between two "mountains". The goal is to drive up the mountain on the right; however, the car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum.

Observation for this domain is car's position and it's velocity. Car's position can be between -1.2 and 0.5 with max velocity 0.07 in either direction. A reward of -1 is received for everytime step. Episode starts with a random position from -0.6 to -0.4 and no velocity.

As the state space is very large, I'll be using a linear approximation for q as $\phi.\theta$ where $\phi$ represents features corresponding to <state,action>pair. Features are obtained from tilecoding. As we approach optimal state-action value function $q^*$, time spend in every episode should decrease. I'll use this to test my agent's ability to learn a good approximation of q. 


Some imports

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from TileCoding import *
from utils import *

EPSILON = 0

Mountain Car Environment

In [11]:
class MountainCar(object):
    class observations(object):
        def __init__(self):
            self.POS_MIN = -1.2
            self.POS_MAX = 0.5
            self.VEL_MIN = -0.07
            self.VEL_MAX = 0.07
            self.high = np.array([self.POS_MAX, self.VEL_MAX])
            self.low = np.array([self.POS_MIN, self.VEL_MIN])

        def reset(self):
            self.pos = np.random.uniform(-0.6, -0.4)
            self.vel = 0.
            return (self.pos, self.vel)

        def act(self, action):
            newVel = self.vel + 0.001 * action - 0.0025 * np.cos(3 * self.pos)
            newVel = min(max(self.VEL_MIN, newVel), self.VEL_MAX)
            newPos = self.pos + newVel
            newPos = min(max(self.POS_MIN, newPos), self.POS_MAX)
            if newPos == self.POS_MIN:
                newVel = 0.0
            self.pos = newPos
            self.vel = newVel

    def __init__(self):
        self.ACT_LEFT = -1
        self.ACT_NONE = 0
        self.ACT_RIGHT = 1
        self.actions = [self.ACT_LEFT, self.ACT_NONE, self.ACT_RIGHT]
        self.observation_space = self.observations()
        self.reset()

    def reset(self):
        self.record =[]
        return self.observation_space.reset()

    def goalReached(self):
        return (self.observation_space.pos == self.observation_space.POS_MAX)

    def step(self, action):
        reward = -1
        self.record.append(([self.observation_space.pos, self.observation_space.vel], action, reward))
        self.observation_space.act(action)
        return ([self.observation_space.pos, self.observation_space.vel],reward, self.goalReached())

Agent based on Semi-gradient SARSA with tile coding for features

In [12]:
class TileCodingAgent(object):
    def __init__(self, env, stepSize, numOfTilings=8, maxSize=2048, n=1):
        self.env = env
        self.maxSize = maxSize
        self.nStep = n
        self.numOfTilings = numOfTilings
        self.stepSize = stepSize / numOfTilings
        self.hashTable = IHT(maxSize)
        self.weights = np.zeros(maxSize)
        # observations need to be scaled
        self.Scale = self.numOfTilings / (env.observation_space.high - env.observation_space.low)

    def getTiles(self, obs, action):
        activeTiles = tiles(self.hashTable, self.numOfTilings, obs*self.Scale, [action])
        return activeTiles

    def value(self, obs, action):
        if obs[0] == self.env.observation_space.POS_MAX:
            return 0.0
        f = self.getTiles(obs, action)
        return np.sum(self.weights[f])

    def learn(self, obs, action, target):
        features = self.getTiles(obs, action)
        estimation = np.sum(self.weights[features])
        delta = self.stepSize * (target - estimation)
        for f in features:
            self.weights[f] += delta

    def getAction(self, obs):
        if np.random.binomial(1, EPSILON) == 1:
            return np.random.choice(self.env.actions)
        values = []
        for action in self.env.actions:
            values.append(self.value(obs, action))
        return argmax(values) - 1

    def generateEpisode(self):
        obs = self.env.reset()
        action = self.getAction(obs)
        done = False
        while not done:
            obs, _, done = self.env.step(action)
            action = self.getAction(obs)

    def semiGradientNStepSarsa(self):
        episode = self.env.record
        T = len(episode)
        for t in range(T):
            ret = 0.
            for i in range(t,min(T,t+self.nStep)):
                ret += episode[i][2]
            if t+self.nStep<T:
                ret += self.value(episode[t+self.nStep][0], episode[t+self.nStep][1])
            self.learn(episode[t][0], episode[t][1], ret)
        return T

Convergence with n-Step returns for targets. I tested with value of 1-step and 4-step returns. As expected, agent with 4-step return converged faster. 

In [13]:
plt.style.use('ggplot')
runs = 5
episodes = 500
numOfTilings = 8
alpha = 0.1
nSteps = [1,4]

env = MountainCar()
timesteps = np.zeros((len(nSteps), episodes))
for run in range(runs):
    for i in range(len(nSteps)):
        agent = TileCodingAgent(env, 0.1, n=nSteps[i])
        for episode in range(0, episodes):
            agent.generateEpisode()
            timesteps[i, episode] += agent.semiGradientNStepSarsa()

timesteps /= runs

for i in range(0, len(nSteps)):
    plt.plot(timesteps[i], label='n-Step Return = '+str(nSteps[i]))
plt.xlabel('Episode')
plt.ylabel('Time Steps per episode')
plt.yscale('log')
plt.legend()
plt.show()

![title](n_step_reward_comparision.png)